In [1]:
import mne
from mne_bids import BIDSPath, read_raw_bids
import numpy as np

In [ ]:
bids_root = "ds005841-download"

subject_id = "001"
task = "regfront"    # lumfront, lumperp, regfront, regperp, signalscreen, signalvr -> Was ist signalscreen, signalvr?

bids_path = BIDSPath(
    subject=subject_id,
    task=task,
    datatype="eeg",
    suffix="eeg",
    extension=".bdf",
    root=bids_root
)

#print(bids_path.fpath)

raw = read_raw_bids(bids_path)
raw.load_data()

#raw.filter(0.1,40, fir_design='firwin')

Extracting EDF parameters from d:\lisa-\Universität_2\Master\2. Semester\EEG\EEG_Project\ds005841-download\sub-001\eeg\sub-001_task-regfront_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from ds005841-download\sub-001\eeg\sub-001_task-regfront_events.tsv.
Reading 0 ... 183807  =      0.000 ...   358.998 secs...


C:\Users\lisa-\AppData\Local\Temp\ipykernel_37176\3322545382.py:17: RuntimeWarning: Did not find any channels.tsv associated with sub-001_task-regfront.

The search_str was "ds005841-download\sub-001\**\eeg\sub-001*channels.tsv"
  raw = read_raw_bids(bids_path)


<RawEDF | sub-001_task-regfront_eeg.bdf, 73 x 183808 (359.0 s), ~102.4 MiB, data loaded>

In [13]:
events = mne.find_events(raw, stim_channel="Status", initial_event=True)
print(events)

Finding events on: Status
130 events found on stim channel Status
Event IDs: [    1     2     3     4    55 65536]
[[     0      0  65536]
 [  3430      0     55]
 [  4201      0      2]
 [  5482      0      4]
 [  6734      0      3]
 [  8111      0      1]
 [ 18481      0      1]
 [ 20076      0      2]
 [ 21306      0      4]
 [ 23296      0      3]
 [ 24599      0      2]
 [ 25789      0      3]
 [ 26916      0      4]
 [ 28117      0      3]
 [ 29655      0      3]
 [ 30753      0      4]
 [ 31983      0      1]
 [ 33063      0      3]
 [ 34241      0      4]
 [ 35385      0      1]
 [ 36512      0      2]
 [ 37638      0      4]
 [ 38708      0      4]
 [ 39835      0      1]
 [ 40893      0      2]
 [ 41985      0      4]
 [ 44049      0      1]
 [ 45833      0      3]
 [ 46954      0      4]
 [ 48047      0      4]
 [ 49265      0      2]
 [ 50380      0      3]
 [ 51438      0      1]
 [ 53737      0      1]
 [ 54875      0      4]
 [ 55950      0      3]
 [ 57071      0      

In [14]:
event_dict = {
    1: "REF_LIGHT",
    2: "REF_DARK",
    3: "RAND_LIGHT",
    4: "RAND_DARK"
}

aggregation = {
    "RAND_LIGHT": "ASYM",
    "RAND_DARK": "ASYM",
    "REF_LIGHT": "SYM",
    "REF_DARK": "SYM"
}

agg_trigger_id = {
    "SYM": 1,
    "ASYM": 2
}

all_events = events

# Keep only triggers that exist in your event_dict
mask = np.isin(all_events[:, 2], list(event_dict.keys()))
events = all_events[mask]

In [17]:
import pandas as pd

onsets = events[:, 0] / raw.info['sfreq']  # convert sample index to seconds
durations = [0.5] * len(events)  # set default duration, adjust if needed
trial_types = [aggregation[event_dict[e]] for e in events[:, 2]]  # map trigger_id to condition
aggregated_trigger_ids = [agg_trigger_id[tt] for tt in trial_types]


df_events = pd.DataFrame({
    "onset": onsets,
    "duration": durations,
    "trial_type": trial_types,
    "trigger_id": aggregated_trigger_ids
})

# Save TSV
output_tsv = r"ds005841-download\sub-001\eeg\sub-001_task-regfront_events_new_new.tsv"
df_events.to_csv(output_tsv, sep="\t", index=False)
print(f"Saved events TSV to {output_tsv}")

Saved events TSV to ds005841-download\sub-001\eeg\sub-001_task-regfront_events_new_new.tsv
